## 영화 추천을 위한 영화 데이터 및 리뷰정보 크롤링

1. 왓차피디아 
- https://pedia.watcha.com/ko-KR

2. 영화개봉정보 - 영화관입장권통합전산망
- https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do#none


In [2]:
# !pip freeze > requirements.txt
!pip install -r requirements.txt

In [1]:
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests


from urllib.parse import quote

import time
#time.sleep을 위한 난수생성
import random

from tqdm import tqdm

# text 전처리
import re

# 마우스 커서 이동을 위한 패키지
import pyautogui

In [2]:
movie = pd.read_excel('data/영화정보 리스트_2024-03-19.xls', header=4)
movie.head(5)

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,남은 인생 10년,The Last 10 Years,2022.0,일본,장편,멜로/로맨스,개봉,후지이 미치히토,NaN
1,달짝지근해: 7510,HONEYSWEET,2022.0,한국,장편,코미디,개봉,이한,(주)무비락
2,폭로,Havana,2022.0,한국,장편,"범죄,스릴러",개봉,홍용호,(주)로그라인스튜디오
3,어디로 가고 싶으신가요,Where Would You Like to Go?,2023.0,한국,장편,드라마,개봉,김희정,(주)인벤트스톤
4,만해 한용운 님의침묵,Silence of Thy,2022.0,한국,장편,드라마,개봉,한명구,주식회사 시네마서울


In [3]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영화명      1547 non-null   object 
 1   영화명(영문)  794 non-null    object 
 2   제작연도     1516 non-null   float64
 3   제작국가     1547 non-null   object 
 4   유형       1547 non-null   object 
 5   장르       1493 non-null   object 
 6   제작상태     1546 non-null   object 
 7   감독       1242 non-null   object 
 8   제작사      632 non-null    object 
dtypes: float64(1), object(8)
memory usage: 108.9+ KB


In [4]:
movie['장르'].fillna('', inplace=True)

C:\Users\tjoeun\AppData\Local\Temp\ipykernel_10360\3068823807.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movie['장르'].fillna('', inplace=True)


In [5]:
movie = movie[~movie['장르'].str.contains('성인')]

In [6]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
Index: 775 entries, 0 to 1546
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   영화명      775 non-null    object 
 1   영화명(영문)  658 non-null    object 
 2   제작연도     753 non-null    float64
 3   제작국가     775 non-null    object 
 4   유형       775 non-null    object 
 5   장르       775 non-null    object 
 6   제작상태     775 non-null    object 
 7   감독       624 non-null    object 
 8   제작사      257 non-null    object 
dtypes: float64(1), object(8)
memory usage: 60.5+ KB


In [7]:
movie['제작연도'].unique()

array([2022., 2023., 2020., 2021., 2004.,   nan, 2019., 1995., 1990.,
       1988., 1964., 1980., 2018., 2015., 1948., 1999., 1996.])

In [8]:
movie['제작연도'] =  movie['제작연도'].fillna(0).apply(lambda x: str(int(np.round(x))))

In [9]:
movie['장르'].unique()

array(['멜로/로맨스', '코미디', '범죄,스릴러', '드라마', '코미디,드라마,멜로/로맨스', '다큐멘터리',
       '사극,드라마', '멜로/로맨스,판타지,드라마', '판타지,드라마', '드라마,멜로/로맨스,범죄',
       '멜로/로맨스,코미디', '드라마,기타', '사극,액션', '드라마,멜로/로맨스,뮤지컬,공연', '애니메이션',
       '드라마,멜로/로맨스', '드라마,사극,스릴러,전쟁', '코미디,드라마', '공포(호러),코미디',
       '미스터리,드라마,스릴러', '드라마,미스터리', '드라마,액션,범죄', 'SF,드라마,스릴러',
       '드라마,판타지,멜로/로맨스', '공연,다큐멘터리', '', '미스터리,범죄,드라마', '스릴러',
       '멜로/로맨스,드라마', '공연', '공포(호러)', '멜로/로맨스,드라마,판타지', '드라마,범죄', '범죄',
       '판타지,어드벤처,애니메이션', '전쟁,액션', '스릴러,멜로/로맨스', '공포(호러),미스터리',
       '액션,어드벤처,판타지', '미스터리,스릴러,드라마', '액션', 'SF,액션', '범죄,액션',
       '공포(호러),스릴러', '코미디,액션', '미스터리,스릴러', '어드벤처,드라마,코미디',
       '애니메이션,어드벤처,판타지', '스릴러,드라마,코미디', '전쟁,드라마', '드라마,멜로/로맨스,가족',
       '액션,드라마,스릴러', '액션,드라마', '애니메이션,어드벤처', '범죄,드라마,스릴러',
       '애니메이션,판타지,어드벤처', '액션,스릴러,전쟁', '미스터리,공포(호러)', 'SF,스릴러',
       '다큐멘터리,드라마,가족', '판타지,액션', '다큐멘터리,범죄', '범죄,액션,스릴러', '스릴러,공포(호러)',
       'SF,액션,드라마', '서부극(웨스턴),코미디,액션', '액션,어드벤처,SF,스릴러', '액션,스릴러',
       '액션,범죄', '드라마,코미디', '

In [10]:
movie_list = movie['영화명'].to_list()
movie_list

['남은 인생 10년',
 '달짝지근해: 7510',
 '폭로',
 '어디로 가고 싶으신가요',
 '만해 한용운 님의침묵',
 '나비효과',
 '울지마 엄마',
 '연악: 나의 운명',
 '어느 날 그녀가 우주에서',
 '3일의 휴가',
 '흠뻑채운 젖은 속살',
 '탐욕에 젖은 섹스',
 '탐욕스런 동침',
 '타락한 유부녀의 유혹',
 '유부녀의 욕망 속살',
 '유부녀의 비밀스런 사정',
 '옆집 유부녀의 속사정',
 '아버님이 채운 유부녀',
 '자우림, 더 원더랜드',
 '위험사회',
 '서울의 봄',
 '싱글 인 서울',
 '콘크리트 유토피아',
 '노량: 죽음의 바다',
 '사랑의 불시착: 라이브 인 서울',
 '두다다쿵: 후후섬의 비밀',
 '나의 피투성이 연인',
 '듣보인간의 생존신고',
 '어른 김장하',
 '돼지의 최후',
 '겨울 이야기',
 '드림',
 '손',
 '비밀',
 '신세계로부터',
 '솔라 플라워',
 '르네에게',
 '익스트림 페스티벌',
 '다음 소희',
 '괴인',
 '돈 룩 디퍼',
 '사랑엔 정의가 없다',
 '그대들은 어떻게 살 것인가',
 '청춘 돼지는 외출하는 여동생의 꿈을 꾸지 않는다',
 '너와 나',
 '말하고 싶은 비밀',
 '상견니',
 '류이치 사카모토: 오퍼스',
 '절해고도',
 '미끼',
 '도티와 영원의 탑',
 '사위 사랑은 장모 4',
 '여배우들 : 바꿔할래?',
 '그녀의 취미생활',
 '라방',
 '오늘 출가합니다',
 '리향',
 '자매의 스와핑',
 '참을 수 없는 탐욕',
 '바지 내린 욕정녀',
 '몸으로 받은 상사',
 '우리 돈 벌레',
 '옆집 아저씨와 같은 침대에 누워 있다',
 '새엄마 잠깐 거긴 안돼',
 '멋대로 들어온 돌싱녀 : 하고 싶어요',
 '남편 말고 의붓아들 : 속궁합이 잘 맞아',
 '쉽게 넣는 그녀 무삭제판',
 '가랑이 채운 처형 무삭제판',
 '르네상스 필름 바이 비욘세',
 '아이엠 장모에요',
 '며느리의 반복 된 숨소리 : 나를 흥분하게 해

In [11]:
year_list = movie['제작연도'].to_list()
year_list

['2022',
 '2022',
 '2022',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2020',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2023',
 '2021',
 '2022',
 '2023',
 '2022',
 '2022',
 '2021',
 '2023',
 '2023',
 '2004',
 '2023',
 '2021',
 '2023',
 '2021',
 '2022',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2023',
 '2021',
 '2023',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2021',
 '2023',
 '2023',
 '2023',
 '2023',
 '0',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '0',
 '2023',
 '2023',
 '2022',
 '2022',
 '2023',
 '2023',
 '2022',
 '2022',
 '2020',
 '2022',
 '2021',
 '2022',
 '2022',
 '2020',
 '2023',
 '2023',
 '2023',
 '2022',
 '2023',
 '2022',
 '2022',
 '2022',
 '2023',
 '2023',
 '2023',
 '2023',
 '2022',
 '2022',
 '2022',
 '2022',
 '2022',
 '2023',
 '2023',
 '2019',
 '2023',
 '2023',
 '2022',
 '2023',
 '2022',
 '2023',
 '2022',
 '2021',
 '2023',
 '2022',
 '2023'

In [14]:
movie.reset_index(drop = True, inplace=True)

In [15]:
movie

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,남은 인생 10년,The Last 10 Years,2022,일본,장편,멜로/로맨스,개봉,후지이 미치히토,NaN
1,달짝지근해: 7510,HONEYSWEET,2022,한국,장편,코미디,개봉,이한,(주)무비락
2,폭로,Havana,2022,한국,장편,"범죄,스릴러",개봉,홍용호,(주)로그라인스튜디오
3,어디로 가고 싶으신가요,Where Would You Like to Go?,2023,한국,장편,드라마,개봉,김희정,(주)인벤트스톤
4,만해 한용운 님의침묵,Silence of Thy,2022,한국,장편,드라마,개봉,한명구,주식회사 시네마서울
...,...,...,...,...,...,...,...,...,...
770,패닉 런,The Desperate Hour,2021,미국,장편,스릴러,개봉,필립 노이스,NaN
771,처음 꽃향기를 만난 순간,Fragrance of the First Flower,2021,대만,장편,"드라마,멜로/로맨스",개봉,덩이한,NaN
772,뮤지컬 번지점프를 하다,NaN,2022,한국,장편,"뮤지컬,공연",개봉,윤영호,씨제이 씨지브이(CJ CGV)(주)
773,뮤지컬 배니싱,NaN,2020,한국,장편,"뮤지컬,공연",개봉,이헌재,씨제이 씨지브이(CJ CGV)(주)


In [16]:
### 데이터 프레임화

base_url = 'https://pedia.watcha.com/ko-KR/searches/movies?query='

movie_info = []
movie_review = []

#for i in range(len(movie_list)):
for i in range(100, 120):
    movie_title = movie['영화명'][i]
    movie_year = movie['제작연도'][i]

    url = base_url + movie_title

    driver = webdriver.Chrome()

    driver.get(url)
    driver.set_window_position(-1800,10)

    # 화면이 뜰때까지 기다리기
    driver.implicitly_wait(10)
    # 창 최대화
    # driver.maximize_window()
    # time.sleep(2.4)
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    #################################################################################################################
    ### 같은 제목의 영화에서 제작연도가 같은 영화를 선택
    same_lis = driver.find_element(By.CLASS_NAME, 'e1493pgd7').find_elements(By.CLASS_NAME, 'e1ic68ft1')

    cnt = 0
    for li in same_lis:
        li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0]
        if li.find_element(By.CLASS_NAME, 'e1gexyxq5').text.strip().split(' ・')[0] != movie_year:
            cnt += 1
        else :
            print(li.find_element(By.CLASS_NAME, 'e1gexyxq4').text.strip())
            print(cnt)
            break

    # 제작연도가 같은 영화가 없을 경우에 첫번째 영화를 선택
    try:
        same_lis[cnt].click()
    except:
        same_lis[0].click()

    #################################################################################################################
    ### 영화 정보 크롤링
    # 영화 소개
    summary = driver.find_element(By.CLASS_NAME, 'e1yew2864').text.strip().replace('\n', '')

    # 감독/출연진
    actor_list = []
    person_lis = driver.find_element(By.CLASS_NAME, 'e1n40jsk1').find_elements(By.CLASS_NAME, 'e4f2vpe2')
    for person in person_lis:
        actor_list.append(person.find_element(By.TAG_NAME, 'a').get_attribute('title'))

    # 전체 평점
    total_rating = driver.find_element(By.CLASS_NAME, 'e15eo6m110').text

    # 전체 평점 리뷰어수
    rating_cnt = driver.find_element(By.CLASS_NAME, 'e15eo6m19').text.split('\n')[-1]
    

    # 정보 1
    info1 = driver.find_element(By.CLASS_NAME, 'e1yew28613').find_elements(By.CLASS_NAME, 'e1yew28611')[1].text

    # 정보 2
    info2 = driver.find_element(By.CLASS_NAME, 'e1yew28613').find_elements(By.CLASS_NAME, 'e1yew28611')[2].text

    time.sleep(round(random.uniform(1.3, 2.2), 3))
    movie_info.append({'title' : movie_title,
                       'movie_num' : i,
                       'summary' : summary,
                       'actor_list': actor_list,
                       'rating' : total_rating,
                       'rating_cnt' : rating_cnt,
                       'info1' : info1,
                       'info2' : info2})
    
    #################################################################################################################
    # 리뷰 페이지 이동
    driver.find_element(By.CLASS_NAME, 'ehapiph0').click()
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    # 스크롤 다운
    for _ in range(5):
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
        driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
        time.sleep(round(random.uniform(1.0, 1.5), 3))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
        time.sleep(round(random.uniform(0.3, 1.2), 3))
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    num = 0
    review_list = driver.find_elements(By.CLASS_NAME, 'egj9y8a4')
    for review_li in review_list:
        
        # 평점이 없는 경우 발생 - ex)보고싶어요
        try:
            # 평점
            review_rating = review_li.find_element(By.CLASS_NAME, 'egj9y8a0').text.strip()

            #사용자명
            reviewer = review_li.find_element(By.CLASS_NAME, 'e10cf2lr1').text.strip()

            # 리뷰
            review = review_li.find_element(By.CLASS_NAME, 'e1hvy88212').text.strip().replace('\n', '')

            # num
            num += 1

            movie_review.append({'title' : movie_title,
                                 'movie_num' : i,
                                 'review_num' : num,
                                 'reviewer' : reviewer,
                                 'review_rating' : review_rating,
                                 'review' : review})
        except:
            continue
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    driver.close()



힐마 아프 클린트 - 미래를 위한 그림
0
트롤: 밴드 투게더
0
아쿠아맨과 로스트 킹덤
0
세계를 뒤흔든 43일
0
제시는 그때 스무살이었다
0
매트
0
매드 하이디
0
라스트 서바이버
0
약속
1
조이랜드
0
인디펜던스 나이트
0


KeyboardInterrupt: 

0           남은 인생 10년
1         달짝지근해: 7510
2                  폭로
3        어디로 가고 싶으신가요
4         만해 한용운 님의침묵
            ...      
1542             패닉 런
1543    처음 꽃향기를 만난 순간
1544     뮤지컬 번지점프를 하다
1545          뮤지컬 배니싱
1546           트루 마더스
Name: 영화명, Length: 775, dtype: object

In [17]:
df_info = pd.DataFrame(movie_info)
df_review = pd.DataFrame(movie_review)

,title,movie_num,summary,actor_list,rating,rating_cnt,info1,info2
0,힐마 아프 클린트 - 미래를 위한 그림,100,”내가 죽은 후 20년 동안 내 작품을 공개하지 말라”는 유언을 남긴 세계 최초의 ...,"[할리나 디르슈카(감독), 힐마 아프 클린트(출연 | 본인), 에른스트 페터 피셔(...",3.6,(171명),"2019 · 다큐멘터리/전기/역사 · 독일, 스웨덴, 스위스, 영국",1시간 34분 · 전체
1,트롤: 밴드 투게더,101,최애 아이돌 그룹 재결합 초읽기! 우리 함께하면 더 좋잖아! \n\n전 세계 모든 ...,"[월트 돈(감독), 팀 하이츠(감독), 안나 켄드릭(성우), 저스틴 팀버레이크(성우...",3.2,"(1,217명)",2023 · 애니메이션/키즈/모험/코미디 · 미국,1시간 32분 · 전체
2,아쿠아맨과 로스트 킹덤,102,아틀란티스 왕국을 이끌 왕의 자리에 오른 ‘아쿠아맨’. 그 앞에 ‘블랙 만타’가 세...,"[제임스 완(감독), 제이슨 모모아(주연 | 아서 커리/아쿠아맨), 패트릭 윌슨(조...",2.8,"(7,634명)",2023 · SF/액션 · 미국,2시간 4분 · 12세
3,화란,103,지옥 같은 현실에서 벗어나고 싶은 소년 '연규'가 조직의 중간 보스 ‘치건’을 만나...,"[김창훈(감독), 홍사빈(주연 | 연규), 송중기(주연 | 치건), 비비(조연 | ...",3.0,"(6,154명)",2023 · 범죄/드라마/스릴러/액션 · 한국,2시간 13분 · 15세
4,세계를 뒤흔든 43일,104,부처님의 행적을 따라서 상월결사 회주 자승 스님을 중심으로 한 인도순례단의 걸음 1...,[이규철(감독)],3.1,(15명),2023 · 다큐멘터리 · 한국,1시간 21분 · 전체
5,제시는 그때 스무살이었다,105,"가난한 어부촌의 벙어리로 태어나,학교도 한번 못가보고 제대로 된 글 한번 깨우쳐보지...","[한명구(감독), 설향(주연 | 제시(금옥)), 임영서(주연 | 브라운 소령), 홍...",1.7,(40명),2022 · 드라마 · 한국,2시간 27분 · 청불
6,매트,106,"매트에 오르면, 심판이 시작된다! \n\n어느날 갑자기 매트 위로 불려온 여덟 명의...","[최지온(감독), 최지온(주연), 하지인(주연), 김장수(주연), 송성사(주연), ...",2.8,(25명),2023 · 미스터리/스릴러 · 한국,2시간 34분 · 12세
7,매드 하이디,107,사악한 독재자가 통치하는 디스토피아의 스위스. 알프스 소녀 ‘하이디’는 할아버지와 ...,"[요하네스 하트만(감독), 앨리스 루시(주연 | 하이디), 맥스 루드링어(주연), ...",2.2,(31명),2022 · 액션/모험/코미디/판타지/공포 · 스위스,1시간 32분 · 청불
8,라스트 서바이버,108,"인류의 마지막 대전쟁 그 이후, 아버지 ‘트로이’와 아들 ‘제이크’는 20년 동안 ...","[드류 마일레아(감독), 알리시아 실버스톤(주연 | 헨리에타), 드류 밴 아커(주연...",3.8,(15명),2021 · SF/액션 · 미국,1시간 38분 · 15세
9,약속,109,엄마와 이별한 아홉 살 ‘시우’는 아빠와 약속한다. \n“1년 뒤에 엄마가 있는 곳...,"[민병훈(감독), 민병훈(출연 | 본인), 민시우(출연 | 본인), 안은미(출연 |...",3.4,(123명),2023 · 다큐멘터리/가족 · 한국,1시간 23분 · 전체


In [ ]:
df_info.to_csv('movie_info.csv', index = False)
df_review.to_csv('movie_review.csv', index = False)

,title,review_num,reviewer,review
0,남은 인생 10년,1,재원,"영화를 보며 눈시울이 붉어졌던 건, 내게도 한땐내가 좀 더 괜찮은 인간이 되고 싶게..."
1,남은 인생 10년,2,박준영 (영화 담다),영롱하고 유려한 피아노와 현악 음악이 흐르고 자연광과 아름다운 풍광이 서두부터 마음...
2,남은 인생 10년,3,오늘은너에게로,"정해진 운명은 야속했지만, 둘의 추억만큼은 행복했기를."
3,남은 인생 10년,4,뭅먼트,"누군가를 살아내게 만드는 것은, 눈앞에서 점점 흐려지는 커다란 희망이 아니라 눈앞에..."
4,남은 인생 10년,5,JY,부슬부슬 봄비처럼 흩날리면 뭐하나결국 2시간동안 내리면흥건해지는건 매한가지인데
...,...,...,...,...
505,3일의 휴가,93,Jamongsoda,"졸았다.. 스팸김치찌개만 생각남..노스페이스, 커피, 소주, 맥주, 스팸, 맛소금…..."
506,3일의 휴가,94,심우영,치유의 대상은 판타지가 아니라 현실이 되어야 한다. 현실을 치유하지 못하는 판타지는...
507,3일의 휴가,95,밀크쉐이크,30분짜리 단편이로 뽑을 소재를 1시간 40분으로 늘려놓은 느낌
508,3일의 휴가,96,박준형,"뻔한 이야기에, 더 뻔한 연출을김해숙, 신민아 두 배우가 살려낸다.엄마와의 <리틀 ..."
